# Дневник

--------------------------------------------

1) Сначала я решил попробовать без генерации новых фичей и без подкрутки параметров запустить LightGBM и посмотреть, что будет. Скор 0.720
При этом пока еще нет ни генерации новых признаков, ни использования таблицы поиска и тд. Попробуем добавить.

2) Просто запускаем код из "baseline_1_pandas.ipynb" и получаем обещаный скор 0.817

3) Оптюнил 10 минут, безрезультатно. Буду придумывать новые признаки. Думаю начать с кластеризации и knn. 

4) Сначала решил просто расширить плавающее окно с 4 до 5 месяцев, результат 0.8192

--------------------------------------------

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pltimport
import matplotlib as mpl, seaborn as sns
import matplotlib.pyplot as plt

# настройка размера шрифта легенды
mpl.rcParams['legend.title_fontsize'] = 15
mpl.rcParams['legend.fontsize'] = 15

import warnings
warnings.filterwarnings('ignore')

In [28]:
df_actions = pd.read_parquet("actions_history/part-00052-554bab9f-fda0-44cc-935e-2114ebd268b9-c000.snappy.parquet")

df_actions.head(5)

,user_id,timestamp,product_id,page_product_id,action_type_id,widget_name_id
0,2346229,2024-03-11 13:35:13,148481529,1.487368e+08,1,6
1,1039167,2024-04-14 18:59:43,167653046,1.391982e+09,1,6
2,2346229,2024-03-27 13:10:44,266202966,1.621779e+08,1,6
3,2774405,2024-04-04 08:47:07,248896971,4.649541e+08,1,6
4,7064286,2024-06-11 19:30:22,321861178,8.577713e+08,1,6


In [29]:
df_search = pd.read_parquet("search_history/part-00000-858f060b-ea82-431a-bbdb-cac1419dd3eb-c000.snappy.parquet")

df_search.head(5)

,user_id,timestamp,search_query,action_type_id,widget_name_id
0,1403825,2024-03-02 19:16:19,колонка алиса,4,23
1,10278224,2024-07-29 12:10:06,батончик,4,23
2,8162261,2024-04-24 06:27:10,hipp combiotic 1,4,23
3,10909038,2024-07-12 04:00:02,бриошь,4,23
4,9148523,2024-04-28 06:58:12,нарезка,4,23


In [30]:
def get_df_info(df: pd.DataFrame, /, thr=0.5, *args, **kwargs) -> pd.DataFrame:
    '''
    Описание:
    Функция выводит статистику о колонках датафрейма в виде датафрейма.
    Результат содержит информацию о типе данных, об уникальных значениях,
    о количестве NaN, нулей и пустых строк,
    о максимальном значении и его частоте,
    два различных примера данных,
    степень адекватности фичи.
    ...

    df: исходный датафрейм,
    thr: граничное значения для опредления адекватность фичи.
    ...

    returns: pd.DataFrame со сборной информацией о датафрейме.
    '''
    
    df_size = df.shape[0]
    columns = df.columns
    numeric_columns = df.select_dtypes(np.number).columns
    string_columns = np.concatenate((df.select_dtypes('string').columns, df.select_dtypes('object').columns))
    result_columns = ["datatype", "nunique", "nan", "zero", "empty_str", "vc_max", "vc_max_freq", "example_1", "example_2", "trash_score"]
    result_dict = dict(zip(result_columns, [[] for _ in range(len(result_columns))]))
    
    for column in columns:
        
        result_dict["datatype"].append(df[column].dtype.name)
        result_dict["nunique"].append(df[column].unique().size)
        
        tmp = df[column].isna().sum()
        push_nan_tmp = tmp / df_size
        push = f"n: {push_nan_tmp: 0.3f}" if tmp > 0 else "-1"
        result_dict["nan"].append(push)

        tmp = column in numeric_columns
        if tmp:
            tmp = df[column].eq(0).sum()
            push_zero_tmp = tmp / df_size
            push = f"z: {push_zero_tmp: 0.3f}" if tmp > 0 else "-1"
        else:
            push_zero_tmp = 0
            push = "-1"
        result_dict["zero"].append(push)

        tmp = column in string_columns
        if tmp:
            tmp = df[column].eq("").sum()
            push_empty_str_tmp = tmp / df_size
            push = f"e: {push_empty_str_tmp: 0.3f}" if tmp > 0 else "-1"
        else:
            push_empty_str_tmp = 0
            push = "-1"
        result_dict["empty_str"].append(push)

        push = df[column].value_counts(dropna=True)
        push_index = push.index[0]
        push_freq = push.values[0]
        result_dict["vc_max"].append(push_index)
        push_vc_max_freq_tmp = push_freq / df_size
        result_dict["vc_max_freq"].append(f"n: {push_vc_max_freq_tmp: 0.3f}")

        tmp = df[column].dropna().unique()
        match tmp.size:
            case 0:
                push = 2 * ["No example"]
            case 1:
                push = [tmp[0], "No example"]
            case _:
                push = [tmp[0], tmp[1]]
        result_dict["example_1"].append(push[0])
        result_dict["example_2"].append(push[1])

        tmp = max(
            push_nan_tmp + push_zero_tmp + push_empty_str_tmp,\
            push_vc_max_freq_tmp if push_vc_max_freq_tmp > thr else 0,
        )
        push = f"{tmp: 0.3f}" if tmp > 0 else "-1"
        result_dict["trash_score"].append(push)

    result_df = pd.DataFrame(result_dict, index=columns).sort_values("trash_score", key=lambda x: -np.float_(x))
    
    return result_df

In [31]:
# Результат

from IPython.display import display

display(get_df_info(df_actions).style.bar(subset=['nunique'], cmap="bwr"))

,datatype,nunique,nan,zero,empty_str,vc_max,vc_max_freq,example_1,example_2,trash_score
widget_name_id,int8,1,-1,-1,-1,6,n: 1.000,6,No example,1.000
action_type_id,int8,3,-1,-1,-1,1,n: 0.872,1,5,0.872
user_id,int32,18114,-1,-1,-1,2346229,n: 0.679,2346229,1039167,0.679
page_product_id,float64,12981,n: 0.000,-1,-1,140119908.000000,n: 0.010,148736804.000000,1391981872.000000,0.000
timestamp,datetime64[ns],106815,-1,-1,-1,2024-04-12 16:01:51,n: 0.000,2024-03-11 13:35:13,2024-04-14 18:59:43,-1
product_id,int64,7307,-1,-1,-1,148481523,n: 0.030,148481529,167653046,-1


In [32]:
# Результат

from IPython.display import display

display(get_df_info(df_search).style.bar(subset=['nunique'], cmap="bwr"))

,datatype,nunique,nan,zero,empty_str,vc_max,vc_max_freq,example_1,example_2,trash_score
action_type_id,int8,1,-1,-1,-1,4,n: 1.000,4,No example,1.000
widget_name_id,int8,1,-1,-1,-1,23,n: 1.000,23,No example,1.000
search_query,object,342031,n: 0.039,-1,e: 0.003,мороженое,n: 0.010,колонка алиса,батончик,0.041
user_id,int32,848876,-1,-1,-1,2346229,n: 0.051,1403825,10278224,-1
timestamp,datetime64[ns],2091382,-1,-1,-1,2024-05-03 11:02:16,n: 0.000,2024-03-02 19:16:19,2024-07-29 12:10:06,-1


In [33]:
df_actions[df_actions['timestamp'] >= '2024-04-28 23:59:59']

,user_id,timestamp,product_id,page_product_id,action_type_id,widget_name_id
4,7064286,2024-06-11 19:30:22,321861178,8.577713e+08,1,6
6,2397776,2024-05-18 13:51:43,749509687,1.423021e+09,1,6
7,2346229,2024-05-23 20:02:53,173187079,2.428874e+08,1,6
8,2346229,2024-06-10 22:06:07,148590542,6.468289e+08,1,6
9,5703773,2024-07-03 11:39:39,1213043165,1.451046e+09,1,6
...,...,...,...,...,...,...
110063,11144632,2024-05-12 11:41:00,1214453307,7.125483e+08,5,6
110064,7914838,2024-07-05 05:00:06,857864096,1.463954e+08,5,6
110065,10861126,2024-07-01 02:04:08,320362167,6.765257e+08,5,6
110066,9228901,2024-05-10 22:08:28,200758355,1.459232e+08,5,6


# Base code

In [34]:
import polars as pl
import pandas as pd
import numpy as np
import catboost
import os
from datetime import date, timedelta

In [35]:
test_start_date = date(2024, 8, 1)
val_start_date = date(2024, 7, 1)
val_end_date = date(2024, 7, 31)
train_end_date = date(2024, 6, 30)
data_path = './'

In [36]:
actions_history = pd.read_parquet(os.path.join(data_path, 'actions_history'))
search_history = pd.read_parquet(os.path.join(data_path, 'search_history'))
product_information = pd.read_csv(os.path.join(data_path, 'product_information.csv'))

In [37]:
pd.read_csv(os.path.join(data_path, 'action_type_info.csv'))

,action_type,action_type_id
0,click,1
1,favorite,2
2,order,3
3,search,4
4,to_cart,5
5,view,6


In [38]:
actions_history = actions_history[actions_history['timestamp'] >= '2024-01-01 00:00:00']
search_history = search_history[search_history['timestamp'] >= '2024-01-01 00:00:00']

In [39]:
val_target = (
    actions_history[
        (actions_history['timestamp'].dt.date >= val_start_date) &
        (actions_history['timestamp'].dt.date <= val_end_date)
    ]
    .assign(has_order=lambda x: (x['action_type_id'] == 3).astype(int))
    .groupby('user_id', as_index=False)
    .agg(target=('has_order', 'max'))
)

# Simple pipeline

## Feats

In [40]:
actions_aggs = {}
actions_id_to_suf = {
    1: "click",
    2: "favorite",
    3: "order",
    5: "to_cart",
}

for id_, suf in actions_id_to_suf.items():
    filtered_data = actions_history[
        (actions_history['timestamp'].dt.date <= train_end_date) &
        (actions_history['timestamp'].dt.date >= train_end_date - timedelta(days=30 * 5)) &
        (actions_history['action_type_id'] == id_)
    ]
    
    merged_data = filtered_data.merge(
        product_information[['product_id', 'discount_price']],
        on='product_id',
        how='left'
    )
    
    aggs = merged_data.groupby('user_id', as_index=False).agg(
        num_products=('product_id', 'count'),
        sum_discount_price=('discount_price', 'sum'),
        max_discount_price=('discount_price', 'max'),
        last_time=('timestamp', 'max'),
        first_time=('timestamp', 'min')
    )
    
    aggs = aggs.rename(columns={
        'num_products': f'num_products_{suf}',
        'sum_discount_price': f'sum_discount_price_{suf}',
        'max_discount_price': f'max_discount_price_{suf}',
        'last_time': f'last_{suf}_time',
        'first_time': f'first_{suf}_time'
    })
    
    aggs[f'days_since_last_{suf}'] = (pd.to_datetime(val_start_date) - aggs[f'last_{suf}_time']).dt.days
    aggs[f'days_since_first_{suf}'] = (pd.to_datetime(val_start_date) - aggs[f'first_{suf}_time']).dt.days
    
    aggs = aggs[[
        'user_id',
        f'num_products_{suf}',
        f'sum_discount_price_{suf}',
        f'max_discount_price_{suf}',
        f'days_since_last_{suf}',
        f'days_since_first_{suf}'
    ]]
    
    actions_aggs[id_] = aggs

In [41]:
# search_aggs
id_ = 4
suf = 'search'

filtered_data = search_history[
    (search_history['action_type_id'] == id_) &
    (search_history['timestamp'].dt.date <= train_end_date) &
    (search_history['timestamp'].dt.date >= train_end_date - timedelta(days=30 * 5))
]
aggs = filtered_data.groupby('user_id', as_index=False).agg(
    num_search=('search_query', 'count'),
    last_search_time=('timestamp', 'max'),
    first_search_time=('timestamp', 'min')
)

aggs = aggs.rename(columns={
    'num_search': f'num_{suf}',
    'last_search_time': f'last_{suf}_time',
    'first_search_time': f'first_{suf}_time'
})

aggs[f'days_since_last_{suf}'] = (pd.to_datetime(val_start_date) - aggs[f'last_{suf}_time']).dt.days
aggs[f'days_since_first_{suf}'] = (pd.to_datetime(val_start_date) - aggs[f'first_{suf}_time']).dt.days

aggs = aggs[[
    'user_id',
    f'num_{suf}',
    f'days_since_last_{suf}',
    f'days_since_first_{suf}'
]]

actions_aggs[id_] = aggs

In [42]:
df = val_target

for _, actions_aggs_df in actions_aggs.items():
    df = df.merge(actions_aggs_df, on='user_id', how='left')

In [43]:
df_pd = df
mask = df_pd.user_id % 10 <= 6

In [44]:
cols = [
    'num_products_click', 
    'sum_discount_price_click', 'max_discount_price_click',
    'days_since_last_click', 'days_since_first_click',
    'num_products_favorite', 'sum_discount_price_favorite',
    'max_discount_price_favorite', 'days_since_last_favorite',
    'days_since_first_favorite', 'num_products_order',
    'sum_discount_price_order', 'max_discount_price_order',
    'days_since_last_order', 'days_since_first_order',
    'num_products_to_cart', 'sum_discount_price_to_cart',
    'max_discount_price_to_cart', 'days_since_last_to_cart',
    'days_since_first_to_cart', 'num_search', 'days_since_last_search',
    'days_since_first_search'
]

In [45]:
train_pool = catboost.Pool(
    df_pd.loc[mask, cols],
    label=df_pd.loc[mask].target,
)
eval_pool = catboost.Pool(
    df_pd.loc[~mask, cols],
    label=df_pd.loc[~mask].target,
)

In [46]:
train_pool.shape, eval_pool.shape

((1311636, 23), (563320, 23))

--------------------------

-----------------------------------------

-----------------------------------------

In [47]:
import optuna
from optuna.samplers import TPESampler

def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'random_strength': trial.suggest_float('random_strength', 1e-3, 10.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_state': 42,
        'eval_metric': 'AUC',
        'loss_function': 'Logloss',
        'task_type': 'CPU',
        'early_stopping_rounds': 50,
    }
    
    model = catboost.CatBoostClassifier(**params)
    model.fit(
        train_pool,
        eval_set=eval_pool,
        verbose=False,
    )
    return model.get_best_score()['validation']['AUC']

# Оптимизация (10 минут)
study = optuna.create_study(
    direction='maximize',
    sampler=TPESampler(seed=42)
)
study.optimize(objective, timeout=600)  # 600 секунд = 10 минут

# Вывод лучших параметров
print("Best trial:")
trial = study.best_trial
print(f"  AUC: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Обучение финальной модели с лучшими параметрами
best_params = {
    'iterations': trial.params['iterations'],
    'depth': trial.params['depth'],
    'learning_rate': trial.params['learning_rate'],
    'l2_leaf_reg': trial.params['l2_leaf_reg'],
    'random_strength': trial.params['random_strength'],
    'bagging_temperature': trial.params['bagging_temperature'],
    'border_count': trial.params['border_count'],
    'random_state': 42,
    'eval_metric': 'AUC',
    'loss_function': 'Logloss',
    'task_type': 'CPU',
}

model = catboost.CatBoostClassifier(**best_params)

[I 2025-03-25 22:32:36,902] A new study created in memory with name: no-name-661d4c26-d4f5-48a0-bd08-a3e967fd5f2e
[I 2025-03-25 22:33:05,423] Trial 0 finished with value: 0.7588615597421022 and parameters: {'iterations': 437, 'depth': 10, 'learning_rate': 0.1205712628744377, 'l2_leaf_reg': 0.24810409748678125, 'random_strength': 0.004207988669606638, 'bagging_temperature': 0.15599452033620265, 'border_count': 45}. Best is trial 0 with value: 0.7588615597421022.
[I 2025-03-25 22:33:31,543] Trial 1 finished with value: 0.7587966500144367 and parameters: {'iterations': 880, 'depth': 8, 'learning_rate': 0.11114989443094977, 'l2_leaf_reg': 0.0012087541473056963, 'random_strength': 7.579479953348009, 'bagging_temperature': 0.8324426408004217, 'border_count': 79}. Best is trial 0 with value: 0.7588615597421022.
[I 2025-03-25 22:33:50,914] Trial 2 finished with value: 0.7568765817460792 and parameters: {'iterations': 263, 'depth': 5, 'learning_rate': 0.028145092716060652, 'l2_leaf_reg': 0.1256

Best trial:
  AUC: 0.7595873345375499
  Params: 
    iterations: 949
    depth: 7
    learning_rate: 0.08503441901354593
    l2_leaf_reg: 1.0104821651596647
    random_strength: 9.056213948070356
    bagging_temperature: 0.4047564375498465
    border_count: 112


-----------------------------------------

-----------------------------------------

-----------------------------------------

In [48]:
model.fit(
    train_pool, 
    eval_set=eval_pool,
    use_best_model=True,
    verbose=10,
    early_stopping_rounds=50,
)

0:	test: 0.7398496	best: 0.7398496 (0)	total: 90.5ms	remaining: 1m 25s
10:	test: 0.7508901	best: 0.7508901 (10)	total: 1.05s	remaining: 1m 29s
20:	test: 0.7523384	best: 0.7523384 (20)	total: 2.01s	remaining: 1m 28s
30:	test: 0.7532984	best: 0.7532984 (30)	total: 2.89s	remaining: 1m 25s
40:	test: 0.7542079	best: 0.7542079 (40)	total: 3.83s	remaining: 1m 24s
50:	test: 0.7548595	best: 0.7548595 (50)	total: 4.73s	remaining: 1m 23s
60:	test: 0.7552981	best: 0.7552981 (60)	total: 5.72s	remaining: 1m 23s
70:	test: 0.7556330	best: 0.7556330 (70)	total: 6.66s	remaining: 1m 22s
80:	test: 0.7558872	best: 0.7558872 (80)	total: 7.55s	remaining: 1m 20s
90:	test: 0.7560793	best: 0.7560793 (90)	total: 8.46s	remaining: 1m 19s
100:	test: 0.7562253	best: 0.7562253 (100)	total: 9.35s	remaining: 1m 18s
110:	test: 0.7564350	best: 0.7564350 (110)	total: 10.3s	remaining: 1m 17s
120:	test: 0.7567019	best: 0.7567019 (120)	total: 11.2s	remaining: 1m 16s
130:	test: 0.7567931	best: 0.7567931 (130)	total: 12.1s	rem

In [49]:
name = 'baseline_1'
model.save_model(f"{name}.bin")

In [50]:
fi = model.get_feature_importance(eval_pool, prettified=True)
fi.head(50)

,Feature Id,Importances
0,sum_discount_price_order,14.477310
1,num_products_order,13.692681
2,max_discount_price_order,11.348042
3,days_since_last_order,9.322558
4,days_since_first_order,6.393884
5,days_since_last_to_cart,5.546367
6,num_products_to_cart,5.111923
7,num_search,5.085638
8,sum_discount_price_to_cart,4.296546
9,num_products_click,3.909231


In [51]:
test_users_submission = (
    pd.read_csv(os.path.join(data_path, 'test_users.csv'))
)

In [52]:
actions_aggs = {}
actions_id_to_suf = {
    1: "click",
    2: "favorite",
    3: "order",
    5: "to_cart",
}

for id_, suf in actions_id_to_suf.items():
    filtered_data = actions_history[
        (actions_history['timestamp'].dt.date <= val_end_date) &
        (actions_history['timestamp'].dt.date >= val_end_date - timedelta(days=30 * 5)) &
        (actions_history['action_type_id'] == id_)
    ]
    
    merged_data = filtered_data.merge(
        product_information[['product_id', 'discount_price']],
        on='product_id',
        how='left'
    )
    
    aggs = merged_data.groupby('user_id', as_index=False).agg(
        num_products=('product_id', 'count'),
        sum_discount_price=('discount_price', 'sum'),
        max_discount_price=('discount_price', 'max'),
        last_time=('timestamp', 'max'),
        first_time=('timestamp', 'min')
    )
    
    aggs = aggs.rename(columns={
        'num_products': f'num_products_{suf}',
        'sum_discount_price': f'sum_discount_price_{suf}',
        'max_discount_price': f'max_discount_price_{suf}',
        'last_time': f'last_{suf}_time',
        'first_time': f'first_{suf}_time'
    })
    
    aggs[f'days_since_last_{suf}'] = (pd.to_datetime(test_start_date) - aggs[f'last_{suf}_time']).dt.days
    aggs[f'days_since_first_{suf}'] = (pd.to_datetime(test_start_date) - aggs[f'first_{suf}_time']).dt.days
    
    aggs = aggs[[
        'user_id',
        f'num_products_{suf}',
        f'sum_discount_price_{suf}',
        f'max_discount_price_{suf}',
        f'days_since_last_{suf}',
        f'days_since_first_{suf}'
    ]]
    
    actions_aggs[id_] = aggs

In [53]:
# search_aggs
id_ = 4
suf = 'search'

filtered_data = search_history[
    (search_history['action_type_id'] == id_) &
    (search_history['timestamp'].dt.date <= val_end_date) &
    (search_history['timestamp'].dt.date >= val_end_date - timedelta(days=30 * 5))
]
aggs = filtered_data.groupby('user_id', as_index=False).agg(
    num_search=('search_query', 'count'),
    last_search_time=('timestamp', 'max'),
    first_search_time=('timestamp', 'min')
)

aggs = aggs.rename(columns={
    'num_search': f'num_{suf}',
    'last_search_time': f'last_{suf}_time',
    'first_search_time': f'first_{suf}_time'
})

aggs[f'days_since_last_{suf}'] = (pd.to_datetime(test_start_date) - aggs[f'last_{suf}_time']).dt.days
aggs[f'days_since_first_{suf}'] = (pd.to_datetime(test_start_date) - aggs[f'first_{suf}_time']).dt.days

aggs = aggs[[
    'user_id',
    f'num_{suf}',
    f'days_since_last_{suf}',
    f'days_since_first_{suf}'
]]

actions_aggs[id_] = aggs

In [54]:
df = test_users_submission

for _, actions_aggs_df in actions_aggs.items():
    df = df.merge(actions_aggs_df, on='user_id', how='left')

In [55]:
df_pd = df

In [56]:
df_pd['predict'] = model.predict(df_pd[cols], prediction_type="Probability")[:, 1]

In [57]:
df_pd[['user_id', 'predict']].to_csv('prep_res.csv', index=False)